# Modelo do transporte - parte 1

## Importação da biblioteca

In [ ]:
import pyomo.environ as pyo

## Definição dos dados de entrada

In [ ]:
custos = [[12, 22, 30], 
          [18, 24, 32], 
          [22, 15, 34]]

capacidade = [100, 140, 160]

demanda = [120, 130, 150]

m = len(capacidade)
n = len(demanda)

## Modelo computacional

In [ ]:
modelo = pyo.ConcreteModel()

### Conjuntos

$I \colon \text{Conjunto de distribuidores,} \; I = \{1,2,\ldots,m\},$

$J \colon \text{Conjunto de consumidores,} \; J = \{1,2,\ldots,n\}.$

In [ ]:
modelo.I = pyo.RangeSet(m)

modelo.J = pyo.RangeSet(n)

### Parâmetros

$c_{ij} \colon \text{Custo unitário de transporte do distribuidor }i \in I \text{ para o consumidor }j \in J,$

$a_i \colon \text{Capacidade de fornecimento do distribuidor }i \in I$,

$b_j \colon \text{Demanda do consumidor }j \in J.$

In [ ]:
modelo.c = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: custos[i-1][j-1])

modelo.a = pyo.Param(modelo.I, initialize=lambda modelo, i: capacidade[i-1])

modelo.b = pyo.Param(modelo.J, initialize=lambda modelo, j: demanda[j-1])

### Variáveis de decisão

$x_{ij} \colon \text{Quantidade a ser transportada do distribuidor }i \in I \text{ para o consumidor }j \in J.$

In [ ]:
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.NonNegativeReals)

### Função objetivo

$\text{min }z(x) = \sum_\limits{i \in I} \sum_\limits{j \in J} c_{ij} x_{ij}.$

In [ ]:
def regra_z(mod):
    return pyo.summation(mod.c, mod.x)

modelo.z = pyo.Objective(rule=regra_z, sense=pyo.minimize) # minimize = default

### Restrições

#### Capacidade

$\sum_\limits{j \in J} x_{ij} = a_i, \;\; \forall i \in I,$

In [ ]:
def regra_capacidade(mod, i):
    return sum(mod.x[i,j] for j in mod.J) <= mod.a[i]

modelo.restr_capacidade = pyo.Constraint(modelo.I, rule=regra_capacidade)

#### Demanda

$\sum_\limits{i \in I} x_{ij} = b_j, \;\; \forall j \in J,$

In [ ]:
def regra_demanda(mod, j):
    return sum(mod.x[i,j] for i in mod.I) >= mod.b[j]

modelo.restr_demanda = pyo.Constraint(modelo.J, rule=regra_demanda)

#### Não negatividade

$x_{ij} \geq 0 \;\; \forall i \in I, j \in J.$

## Impressão do modelo

In [ ]:
modelo.pprint()

## Resolução

In [ ]:
resultado = pyo.SolverFactory('glpk').solve(modelo)

In [ ]:
resultado.write()

In [ ]:
modelo.x.pprint()

In [ ]:
modelo.z()